# 正则表达式

查找复合特定模式的文本。

## 1 查找文本模式

假设我们想要查找一个电话号码，格式为：xxx-xxxx-xxxx.

In [7]:
def phone_number_validator(phone_number):
    if len(phone_number) != 13:
        return False
    
    for i in range (0, 3):
        if not phone_number[i].isdecimal():
            return False
    
    if phone_number[3] != '-':
        return False
    
    for i in range(4, 8):
        if not phone_number[i].isdecimal():
            return False
    
    if phone_number[8] != '-':
        return false
    
    for i in range(9, 13):
        if not phone_number[i].isdecimal():
            return False

    return True

In [8]:
phone_number_validator('136-1122-1212')

True

In [9]:
phone_number_validator('136-112-21212')

False

In [10]:
phone_number_validator('1361-1221-212')

False

In [11]:
phone_number_validator('021-21029871')

False

如果想在很长的一句话中查找复合格式的字符串，我们需要循环去做。

In [20]:
def find_valid_phone_number(phone_number):
    is_found = False
    for i in range(len(message)):
        message_slice = message[i:i+13] # string slicing 超出范围后没有影响
        if phone_number_validator(message_slice):
            is_found = True
            print('Find a valide phone number: ', message_slice)

    if not is_found:
        print('Nothing found.')

In [23]:
import time

In [24]:
message = 'Hello, my phone number is 136-1122-1212'

start_time = time.time()
find_valid_phone_number(message)
end_time = time.time()
print(end_time - start_time)

Find a valide phone number:  136-1122-1212
0.00041031837463378906


In [22]:
message = 'Hello, my phone number is'

find_valid_phone_number(message)

Nothing found.


## 2. 使用正则表达式查找文本

In [4]:
import re

### 2.1 写正则表达式字符串

首先，我们需要写regex，上述的手机号的format 可以看成：
- 3个数字 `\d{3}`
- 1个短横杠 `-`
- 4个数字 `\d{4}`
- 1个短横杠 `-`
- 4个数字 `\d{4}`

所以连接起来就是：

`'\d{3}-\d{4}-\d{4}'`

注意，这里实际应该是`\\d`, 但是因为是raw string，所以只需要一个`\`即可。

### 2.2 compile 正则表达式字符串，创建regex 对象

返回一个regex模式对象

In [32]:
valide_phone_number_regex = re.compile(r'\d{3}-\d{4}-\d{4}')

### 2.3 匹配regex 对象

regex 对象的`search()` 方法接受一个字符串作为参数，查找该正则表达式的所有匹配的子字符串。
- 如果没有找到，则返回None。
- 如果找到了，则返回一个Match 对象。Match 对象有一个group 方法，返回所有匹配的字符串。

`search()` 方法只返回第一次匹配的文本，如果想找匹配的所有文本，可以使用`findall()` 方法（后面会介绍）。

In [37]:
message = 'Hello, my phone number is 136-1122-1212.'
# message = 'Hello, my phone number is 136-1122-1212. You can also call me via: 188-1111-2222.'

start_time = time.time()
mo = valide_phone_number_regex.search(message)
print(mo.group())
end_time = time.time()
print(end_time - start_time)

136-1122-1212
0.00045680999755859375


In [38]:
message2 = 'Hello, I do not have a mobile phone.'
mo = valide_phone_number_regex.search(message2)
print(mo.group()) 

AttributeError: 'NoneType' object has no attribute 'group'

从上面的例子可以看出来，因为没有找到，所以Match 对象是一个None，因此无法调用`group()` 方法。

下面我们来总结以下使用regex 查找字符串的一般步骤：
- 编写regex 字符串
- 通过regex 字符串创建regex 对象(`re.compile`)
- 调用regex 对象的search 方法，查找字符串，返回Match 对象
- 判断Match 对象是否为空，如果不为空，调用`group()` 方法获取匹配结果

下面，我们介绍一些regex 的高级用法

## 3. Regex 高级用法

### 3.1 利用括号分组

在前面的例子里，我们想把前三位提取出来，来判断运营商。

In [5]:
valide_phone_number_group_regex = re.compile(r'(\d{3})-(\d{4}-\d{4})')  # 第一个括号是第一组，第二个括号是第二组

In [6]:
message = 'Hello, my phone number is 136-1122-1212.'

mo = valide_phone_number_group_regex.search(message)
print(mo.group(1))
if int(mo.group(1)) > 134 and int(mo.group(1)) < 140:
    print('Operator: China Mobile')
    
print(mo.group(2))
print(mo.group(1))
print(mo.group(0))  # 传入0 或无参数，返回整个匹配文本
print(mo.group())

136
Operator: China Mobile
1122-1212
136
136-1122-1212
136-1122-1212


#### 使用`groups()` 方法一次返回所以匹配分组

返回值是一个tuple

In [51]:
mo.groups()

('136', '1122-1212')

In [52]:
type(mo.groups())

tuple

可以看出，括号在正则表达式中有特殊的意义，所以如果在文本中需要匹配括号，需要带转移字符(`'\('`)。

In [68]:
message2 = 'my fix phone is: (021) 2121-2121.'

fix_phone_pattern = r'\((\d{3})\) (\d{4}-\d{4})'
fix_phone_regex = re.compile(fix_phone_pattern)
mo = fix_phone_regex.search(message2)
print('Province code: ', mo.group(1))
print('Phone number: ', mo.group(2))

Province code:  021
Phone number:  2121-2121


### 3.2 用管道匹配多个分组

当我们希望匹配的表达式多于一个时，我们使用管道`|`。

例如，在上面的例子里，有的省的区号是四位数字，所以我们可以使用管道，来匹配多种情况。

In [70]:
message1 = 'my fix phone is: (021) 2121-2121.'
message2 = 'my fix phone is: (0991) 2121-2121.'

fix_phone_pattern = r'\((\d{3}|\d{4})\) (\d{4}-\d{4})'
fix_phone_regex = re.compile(fix_phone_pattern)
mo = fix_phone_regex.search(message2)
print('Province code: ', mo.group(1))
print('Phone number: ', mo.group(2))

Province code:  0991
Phone number:  2121-2121


In [71]:
mo = fix_phone_regex.search(message1)
print('Province code: ', mo.group(1))
print('Phone number: ', mo.group(2))

Province code:  021
Phone number:  2121-2121


也可以使用管道，来做lemmatization，相同前缀的词可以识别成一样的。

In [86]:
suffix_regex = re.compile(r'process(ing|or|ed|es|)')  # 注意，空字符要放在最后

In [83]:
mo = suffix_regex.search('this is a processor')
mo.group()

'processor'

In [84]:
mo = suffix_regex.search('He is processing the document')
mo.group()

'processing'

In [85]:
mo = suffix_regex.search('pls. follow the process.')
mo.group()

'process'

### 3.3 用问号实现可选匹配（零次或一次）

有的时候，有的匹配是可选的，我们需要用问号`?`来表明它前面的pattern是可选的，即可以出现零次或一次。

例如，固定电话，我们可以写成:
- (021)21212121
- (021) 21212121
- (021) 2121-2121
- (021)2121-2121

我们看出，区号和电话号码之间的空格，以及电话号码前四位和后四位的横线是可选的。因此，我们修改之前的表达式，如下所示。

In [90]:
fix_phone_regex = re.compile(r'\((\d{3})\) ?(\d{4}-?\d{4})')  # 空格和-均为可选匹配

In [91]:
message = 'my fix phone is: (021) 2121-2121.'
mo = fix_phone_regex.search(message)
print('Province code: ', mo.group(1))
print('Phone number: ', mo.group(2))

Province code:  021
Phone number:  2121-2121


In [92]:
message = 'my fix phone is: (021)2121-2121.'
mo = fix_phone_regex.search(message)
print('Province code: ', mo.group(1))
print('Phone number: ', mo.group(2))

Province code:  021
Phone number:  2121-2121


In [93]:
message = 'my fix phone is: (021)21212121.'
mo = fix_phone_regex.search(message)
print('Province code: ', mo.group(1))
print('Phone number: ', mo.group(2))

Province code:  021
Phone number:  21212121


In [94]:
message = 'my fix phone is: (021) 21212121.'
mo = fix_phone_regex.search(message)
print('Province code: ', mo.group(1))
print('Phone number: ', mo.group(2))

Province code:  021
Phone number:  21212121


因此，如果需要匹配一个真正的问号，需要使用转义字符。

### 3.4 用星号匹配零次或多次

In [95]:
fix_phone_regex = re.compile(r'\((\d{3})\) *(\d{4}-*\d{4})')  # 空格和-均可出现任意次

In [98]:
# 多次
message = 'my fix phone is: (021)    2121--2121.'
mo = fix_phone_regex.search(message)
print('Province code: ', mo.group(1))
print('Phone number: ', mo.group(2))

Province code:  021
Phone number:  2121--2121


In [99]:
# 零次
message = 'my fix phone is: (021)21212121.'
mo = fix_phone_regex.search(message)
print('Province code: ', mo.group(1))
print('Phone number: ', mo.group(2))

Province code:  021
Phone number:  21212121


### 3.5 用加号匹配一次或多次


In [100]:
fix_phone_regex = re.compile(r'\((\d{3})\) +(\d{4}-+\d{4})')  # 空格和-均可出现任意次

In [101]:
# 多次
message = 'my fix phone is: (021)    2121--2121.'
mo = fix_phone_regex.search(message)
print('Province code: ', mo.group(1))
print('Phone number: ', mo.group(2))

Province code:  021
Phone number:  2121--2121


In [106]:
# 零次，匹配失败
message = 'my fix phone is: (021)21212121.'
mo = fix_phone_regex.search(message)
mo == None

True

### 3.6 用花括号匹配特定次数

这个我们之前讲过了，使用`\d\d\d`匹配三个数字，等同于使用`\d{3}`。

我们可以匹配一个范围，例如`\d{3,4}`可以匹配三个或四个数字。我们可以修改之前区号匹配的regex.


In [107]:
message1 = 'my fix phone is: (021) 2121-2121.'
message2 = 'my fix phone is: (0991) 2121-2121.'

fix_phone_pattern = r'\((\d{3,4})\) (\d{4}-\d{4})'
fix_phone_regex = re.compile(fix_phone_pattern)

mo = fix_phone_regex.search(message1)
print('Province code: ', mo.group(1))
print('Phone number: ', mo.group(2))

mo = fix_phone_regex.search(message2)
print('Province code: ', mo.group(1))
print('Phone number: ', mo.group(2))

Province code:  021
Phone number:  2121-2121
Province code:  0991
Phone number:  2121-2121


- `\d{3,}`: 匹配至少三个数字
- `\d{,3}`: 匹配最多三个数字

In [108]:
ha_pattern = r'(ha){,5}'
ha_regex = re.compile(ha_pattern)

mo = ha_regex.search('haha')
mo.group()

'haha'

In [109]:
mo = ha_regex.search('hahahahaha')
mo.group()

'hahahahaha'

In [110]:
mo = ha_regex.search('hahahahahaha')  # 只能匹配5个ha
mo.group()

'hahahahaha'

### 3.7. 贪心匹配和非贪心匹配

在上面的hahaha 的例子中，默认是贪心匹配的，即如果能匹配五个，就返回五个，否则返回4个，三个，...

python 的regex，默认是贪心的，在二义的情况下，会尽可能匹配最常的字符串。

如果想用非贪心版本，可以在花括号后加一个问号。

In [111]:
ha_pattern = r'(ha){3,5}?'
ha_regex = re.compile(ha_pattern)
mo = ha_regex.search('hahahahaha')
mo.group()

'hahaha'

注意，问号`?` 在正则表达式中有两个含义：
- 可选的（出现0或1次）
- 非贪心（跟在次数后）

### 3.8 `findall()` 方法

与`search()`方法不同，`findall()` 方法返回一组字符串，包含所有匹配的字符串。

In [112]:
message = 'Hello, my phone number is 136-1122-1212. You can also call me via: 188-1111-2222.'

valide_phone_number_group_regex = re.compile(r'\d{3}-\d{4}-\d{4}') 

mo = valide_phone_number_group_regex.search(message)
mo.group()

'136-1122-1212'

In [113]:
phone_numbers = valide_phone_number_group_regex.findall(message)
print(phone_numbers)

['136-1122-1212', '188-1111-2222']


如果正则表达式中有分组，`findall()` 方法将返回一个tuple 的列表。

In [114]:
valide_phone_number_group_regex = re.compile(r'(\d{3})-(\d{4}-\d{4})') 
phone_numbers = valide_phone_number_group_regex.findall(message)
print(phone_numbers)

[('136', '1122-1212'), ('188', '1111-2222')]


## 4. 字符的分类

前面我们使用`\d` 来代表一个数字，也就是说，`\d` 等同于 `(0|1|2|3|4|5|6|7|8|9)`。

`\d` 是一个缩写字符类，下面我们来看一下还有什么其他的分类。

| 缩写字符类      | 表示          |
| ------------- |:-------------:| 
| \d            | 0-9 任何数字字符                        |
| \D            | 除0-9以外的任何字符                      |
| \w            | 任何字母，数字，或下划线（可以认为是匹配单词）|
| \W            | 除字母，数字，下划线以外的任何字符          |
| \s            | 空格，制表符或换行符                      |
| \S            | 除空格，制表符或换行符以外的任何字符        |

例如，我们可以使用以下regex 类匹配key-value pair(其中key 是数字，value 是单词)。

In [116]:
text = '23:Jordan, 33:Pipen, 30:Currey, coach:Jackson'

key_value_pair_regex = re.compile(r'\d+:\w+')
key_value_pair_regex.findall(text)

['23:Jordan', '33:Pipen', '30:Currey']

从上面我们可以看出，成功找出了球员和相应的号码。

### 4.1 建立自己的字符分类

例如，我们只想匹配元音，可以将自己想要匹配的字符放在方括号中。注意，方括号中不需要转移字符，因为所有正则表达式符号都不会被解释。

In [117]:
vowel_regex = re.compile(r'[aeiouAEIOU]')

In [119]:
text = '23:Jordan, 33:Pipen, 30:Currey, coach:Jackson'
vowel_regex.findall(text)  # 找出一句话中所有的元音

['o', 'a', 'i', 'e', 'u', 'e', 'o', 'a', 'a', 'o']

### 4.2 非字符类

在左方括号前加上`^`, 就可以得到非字符类。

In [123]:
nonvowel_regex = re.compile(r'[^aeiouAEIOU]')  # 非元音
text = '23:Jordan, 33:Pipen, 30:Currey, coach:Jackson'
print(nonvowel_regex.findall(text))  # 找出一句话中所有的非元音字符（包括辅音和特殊字符）

['2', '3', ':', 'J', 'r', 'd', 'n', ',', ' ', '3', '3', ':', 'P', 'p', 'n', ',', ' ', '3', '0', ':', 'C', 'r', 'r', 'y', ',', ' ', 'c', 'c', 'h', ':', 'J', 'c', 'k', 's', 'n']


### 4.3 插入字符和美元字符

- 插入字符 `^` 表明匹配必须发生在文本开始处
- 美元字符 `$` 表明匹配必须发生在文本结束处
- 可以同时使用插入字符和美元字符

In [129]:
hello_regex = re.compile(r'^Hello \w+!$')  # 匹配整个字符串，成功

text = 'Hello Michael!'
print(hello_regex.search(text))

<_sre.SRE_Match object; span=(0, 14), match='Hello Michael!'>


In [130]:
hello_regex = re.compile(r'^Hello \w+!$')  # 匹配整个字符串，失败

text = 'Hello Michael! How are you?'
print(hello_regex.search(text))

None


In [132]:
hello_regex = re.compile(r'^Hello \w+!')  # 从字符串开始的地方匹配，成功

text = 'Hello Michael! How are you?'
print(hello_regex.search(text))

<_sre.SRE_Match object; span=(0, 14), match='Hello Michael!'>


### 4.4 通配字符

#### 句点`.` 匹配除了换行符之外的所有字符。
因此，如果要匹配句号，需要转义`\.`.

In [134]:
at_regex = re.compile(r'.at')
at_regex.findall('the cat in the hat sat on the flat mat')

['cat', 'hat', 'sat', 'lat', 'mat']

#### 点星 `.*` 匹配所有字符 

注意，这里点星使用贪婪模式，匹配尽可能多的文本。如果要使用非贪婪模式，可以使用`.*?`

In [136]:
text = '<some thing here> this is outside>'

# 一个左尖括号，任意字符，右尖括号，贪心匹配。
greedy_regex = re.compile(r'<.*>')

# 一个左尖括号，任意字符，右尖括号，非贪心匹配。
non_greedy_regex = re.compile(r'<.*?>')

mo_greedy = greedy_regex.search(text)
print(mo_greedy.group())

mo_non_greedy = non_greedy_regex.search(text)
print(mo_non_greedy.group())

<some thing here> this is outside>
<some thing here>


#### `re.DOTALL` 匹配换行符

将 `re.DOTALL` 作为`re.compile()` 的第二个参数，就可以让dot 匹配所有字符(包括换行符)。

In [139]:
text = 'the cat in the hat sat on the flat mat.\nat this moment, my mum come back.'

at_regex = re.compile(r'.at')
at_regex.findall(text)

['cat', 'hat', 'sat', 'lat', 'mat']

In [140]:
at_regex = re.compile(r'.at', re.DOTALL)
at_regex.findall(text)

['cat', 'hat', 'sat', 'lat', 'mat', '\nat']

#### `re.IGNORECASE` 或 `re.I` 不区分大小写

In [141]:
text_upper = text.upper()
text_upper

'THE CAT IN THE HAT SAT ON THE FLAT MAT.\nAT THIS MOMENT, MY MUM COME BACK.'

In [142]:
at_regex = re.compile(r'.at')
at_regex.findall(text_upper)

[]

In [143]:
at_regex = re.compile(r'.at', re.I)
at_regex.findall(text_upper)

['CAT', 'HAT', 'SAT', 'LAT', 'MAT']

#### 组合使用

如果既想使用DOTALL, 又想使用IGNORECASE, `re.compile()` 只接受一个值作为第二参数。

这里，我们可以使用管道，将变量组合起来。

In [144]:
at_regex = re.compile(r'.at', re.DOTALL | re.I)

# \nAt
text = 'the cat in the hat sat on the flat mat.\nAt this moment, my mum come back.'
at_regex.findall(text)

['cat', 'hat', 'sat', 'lat', 'mat', '\nAt']

### 4.5 用`sub()`方法替换字符串

regex 对象的`sub()` 方法有两个参数：
- 第一个参数是匹配后替换的字符串
- 第二个参数是被替换的字符串

In [149]:
name_regex = re.compile(r'(Mr.|Ms.|Miss) (\w+)')

text = 'Good morning, Mr. Wang. How are you?'
mo = name_regex.search(text)
mo.group(2)

'Wang'

下面，我们需要隐藏名字。

In [152]:
name_regex.sub(r'\1***', text)  # 注意，这里是一个raw string

'Good morning, Mr.***. How are you?'

我们发现，`\1` 保留了匹配的第一部分，把剩余部分替换成`***`

接下来，我们只想隐藏称呼(性别)，可以显示姓名。

In [157]:
name_regex.sub(r'(title)\2', text)

'Good morning, (title)Wang. How are you?'

### 4.6 管理复杂的正则表达式

当正则表达式很长的时候，可以将正则表达式放在多行中(使用多行字符串`"""..."""`)，并加上注释。然后使用`re.VERBOSE` 作为第二个参数，忽略字符串中的空白符和注释。

这样做的目标是增加可读性。对于很复杂，很长的regex 通常使用这种方法。

In [146]:
fix_phone_regex = re.compile(r'''\((\d{3,4})\)  # 区号
                        \s                      # 空格符
                        (\d{4}-\d{4})           # 电话号码
                        ''', re.VERBOSE)        

In [147]:
message = 'my fix phone is: (021) 2121-2121.'

mo = fix_phone_regex.search(message)
print('Province code: ', mo.group(1))
print('Phone number: ', mo.group(2))

Province code:  021
Phone number:  2121-2121


### 4.7 正则表达式符号复习

| 正则表达式      | 意义          |
| ------------- |:-------------:| 
| ？            | 0次或1次                        |
| +            | 1次或多次                     |
| *            | 任意次（0次，1次，多次）|
| {n}            | 匹配n次          |
| {n,}            | 匹配至少n次                      |
| {,n}           | 匹配最多n次        |
| {n,m}            | 匹配至少那次，至多m次                        |
| {n,m}? / *？/ +？           | 非贪心匹配                      |
| ^regex            | 从头匹配|
| regex$            | 从尾匹配          |
| .            | 匹配所有字符，除了换行符                      |
| \d, \w, \s           | 匹配数字，单词和空格        |
| \D, \W, \S            | 除数字，单词和空格以外          |
| [abc]            | 匹配方括号内的任意字符                    |
| [^abc]           | 匹配除了方括号内任意字符        |

## 5. Excercises

## 5.1 Email 和电话号码提取